## План

* Удалить выбросы
* Удалить редкие категори
* Логарифмирование трат
* tf-idf для категориальных фичей
* Разбиение операций на траты и пополнения

In [124]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import roc_auc_score
from time import time
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [301]:
transactions = pd.read_csv('data/transactions.csv')

genders = pd.read_csv('data/train.csv', encoding = "utf-8")
genders = genders.drop('Unnamed: 0', axis = 1)

trans_types = pd.read_csv('data/trans_types.csv', encoding = "utf-8", sep=';')

mcc_codes = pd.read_csv('data/mcc_codes.csv', sep=';')

In [302]:
transactions.head(5)

,client_id,trans_time,mcc_code,trans_type,amount,term_id,trans_city
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,35 08:24:41,4829,2370,-1808.56,NaN,Tver
1,81c7349f7bdcb661bc7d5f5f0fdac6aa,351 13:27:15,6011,7010,7954.51,442207,Kazan
2,fc0c7c937f139bd6d1c5b5d0a0023dd3,150 16:03:59,6011,2010,-12293.56,NaN,Khabarovsk
3,3495487d5beaa5caaa0682fddc38379a,373 11:54:12,6010,7070,49.52,888987,Kaliningrad
4,07e279042b1da23710fa2f65034d30b9,340 13:11:45,6010,7070,45.29,888918,Saint Petersburg


In [303]:
def apply_log(x):
    if x > 0:
        return np.log(1+x)
    return -np.log(1-x)

transactions['amount_log'] = transactions['amount'].apply(apply_log)

In [304]:
def work_relax_hours(s):
    data = str(s).split()
    day = int(data[0])
    time = data[1]
    h, m, s = map(int, time.split(':'))
    return int(25200 < h * 3600 + m * 60 + s < 61200)


def time_to_hours(s):
    data = str(s).split()
    time = data[1]
    h, m, s = map(int, time.split(':'))
    return h

def time_to_day(s):
    data = str(s).split()
    day = int(data[0])
    return day % 7

def is_holiday(n):
    if n == 5 or n == 6: return 1
    return 0

In [305]:
def generate_tfidf_features(df, col):
    bot = df.groupby('client_id')[col].apply(lambda x: list(x))
    
    corpus = [' '.join(map(str, x)) for x in bot]
    
    tfidf = TfidfVectorizer()
    tfidf.fit(corpus)

    tfidf_features = pd.DataFrame(
        tfidf.transform(corpus).A,
        columns=[f'tfidf_{col}_{k}' for k in tfidf.get_feature_names_out()]
    )
    tfidf_features['client_id'] = bot.index

    return tfidf_features


# def generate_one_hot_features(df, col):
#     return pd.get_dummies(transactions[['client_id', col]], columns=[col], dtype=int).groupby('client_id').sum()

In [306]:
def generate_pivot_features(df, col, value, aggfuncs):
    pivot = df.pivot_table(values=[value], index=['client_id'], columns=[col], aggfunc=aggfuncs)
    pivot.columns = [f'{col}_{lst[-1]}_{value}_{aggfuncs if len(lst) < 3 else lst[0]}' for lst in pivot.columns]
    return pivot.reset_index()

In [307]:
transactions['work_relax'] = transactions['trans_time'].apply(work_relax_hours)
transactions['day'] = transactions['trans_time'].apply(time_to_day)
transactions['is_holiday'] = transactions['day'].apply(is_holiday)
transactions['hour'] = transactions['trans_time'].apply(time_to_hours)

transactions.drop(['trans_time', 'term_id'], axis=1, inplace=True)

In [85]:
# transactions = transactions[(transactions['amount'] > transactions['amount'].quantile(0.01)) & (transactions['amount'] < transactions['amount'].quantile(0.99))]

In [308]:
transactions.loc[transactions['amount'] > 0, 'amount_pos'] = transactions['amount']
transactions.loc[transactions['amount'] <= 0, 'amount_neg'] = transactions['amount']

In [309]:
transactions

,client_id,mcc_code,trans_type,amount,trans_city,amount_log,work_relax,day,is_holiday,hour,amount_pos,amount_neg
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,4829,2370,-1808.56,Tver,-7.500839,1,0,0,8,NaN,-1808.56
1,81c7349f7bdcb661bc7d5f5f0fdac6aa,6011,7010,7954.51,Kazan,8.981620,1,1,0,13,7954.51,NaN
2,fc0c7c937f139bd6d1c5b5d0a0023dd3,6011,2010,-12293.56,Khabarovsk,-9.416912,1,3,0,16,NaN,-12293.56
3,3495487d5beaa5caaa0682fddc38379a,6010,7070,49.52,Kaliningrad,3.922369,1,2,0,11,49.52,NaN
4,07e279042b1da23710fa2f65034d30b9,6010,7070,45.29,Saint Petersburg,3.834926,1,4,0,13,45.29,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3563524,e91858185ae43cf3970220748893a01b,6010,7070,227.40,Penza,5.431098,1,6,1,13,227.40,NaN
3563525,429b6c8255f13bdb506fdbd6e342ccb1,4814,1030,-1084.44,Khabarovsk,-6.989741,0,3,0,19,NaN,-1084.44
3563526,9443b6a0d948cf368d8d72eb295bdfdf,5411,1110,-569.20,Tver,-6.345987,0,6,1,23,NaN,-569.20
3563527,1be75d38ad89c0f2b4d85b95f0fd8069,5814,1110,-71.43,Vladivostok,-4.282621,0,3,0,23,NaN,-71.43


In [565]:
tfidf_features_mcc = generate_tfidf_features(transactions, 'mcc_code')
tfidf_features_type = generate_tfidf_features(transactions, 'trans_type')

mcc_neg_amount = generate_pivot_features(transactions, 'mcc_code', 'amount_neg', ['mean'])
mcc_pos_amount = generate_pivot_features(transactions, 'mcc_code', 'amount_pos', ['mean'])

type_neg_amount = generate_pivot_features(transactions, 'trans_type', 'amount_neg', ['mean'])
type_pos_amount = generate_pivot_features(transactions, 'trans_type', 'amount_pos', ['mean'])


day_of_week1_features = generate_pivot_features(transactions, 'day', 'amount_pos', ['mean'])
day_of_week2_features = generate_pivot_features(transactions, 'day', 'amount_neg', ['mean'])
day_of_week_features = generate_pivot_features(transactions, 'day', 'amount', ['mean'])
hour_features = generate_pivot_features(transactions, 'hour', 'amount', ['mean'])

In [566]:
amount_agg = transactions.groupby('client_id').agg({
    'amount_pos': ['mean', 'std', 'max', 'min'],
    'amount_neg': ['mean', 'std', 'max', 'min'],
    'amount': ['mean', 'std'],
}).reset_index()

amount_agg.columns = [f'{head}_{stat}' if stat != '' else head for head, stat in amount_agg.columns]

In [602]:
# merge features
data = (
    tfidf_features_mcc  
    
    .merge(mcc_neg_amount)
    .merge(mcc_pos_amount)
    
    .merge(type_neg_amount)
    .merge(type_pos_amount)
    
    .merge(day_of_week_features) 
    .merge(tfidf_features_type)
    .merge(hour_features)
    
    .merge(amount_agg)
).fillna(0).merge(genders, how='left')

In [603]:
data.drop(['client_id'], inplace=True, axis=1)

In [604]:
# corr = data.corr()
# high_corr = (abs(np.tril(corr)) - np.identity(corr.shape[0]) >= 0.9).nonzero()
# data = data.drop(corr.columns[high_corr[0]], axis=1)

In [605]:
data.shape

(7704, 732)

In [606]:
X_final_test = data[data['gender'].isna()].drop(['gender'], axis=1)
X = data[~data['gender'].isna()]
y = X['gender']
X.drop(['gender'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.6, random_state=0)

C:\Users\alexe\AppData\Local\Temp\ipykernel_23964\2254477700.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(['gender'], axis=1, inplace=True)


In [607]:
y_train.shape, y_val.shape, y_test.shape

((5545,), (832,), (555,))

In [620]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_normed = scaler.transform(X)
X_train_normed = scaler.transform(X_train)
X_val_normed = scaler.transform(X_val)
X_test_normed = scaler.transform(X_test)
X_final_test_normed = scaler.transform(X_final_test)

In [609]:
#ааааааааааааааа нахуй ML наконец то

In [610]:
xgb = XGBClassifier(
    eval_metric="auc",
    n_estimators=10000,
    random_state=42, 
    verbosity=0, 
    eta = 0.04, 
    max_depth = 3, 
    gamma = 0.9,
    min_child_weight=20,
    colsample_bytree=0.3,
    colsample_bynode=0.2,
    subsample =0.9,
    reg_lambda=0.6,
    # reg_alpha=0.3,
    early_stopping_rounds=100
)


xgb.fit(
    X_train_normed, y_train, 
    eval_set=[[X_val_normed, y_val]], 
    verbose=50
)


xgb_test_preds = xgb.predict_proba(X_test_normed,
                              iteration_range=(0, xgb.best_iteration + 1))
xgb_val_preds = xgb.predict_proba(X_val_normed,
                              iteration_range=(0, xgb.best_iteration + 1))
xgb_train_preds = xgb.predict_proba(X_train_normed,
                               iteration_range=(0, xgb.best_iteration + 1))

print('test score', roc_auc_score(y_test, [i[1] for i in xgb_test_preds]))
print('val score', roc_auc_score(y_val, [i[1] for i in xgb_val_preds]))
print('train score', roc_auc_score(y_train, [i[1] for i in xgb_train_preds]))

[0]	validation_0-auc:0.60653
[50]	validation_0-auc:0.83253
[100]	validation_0-auc:0.84966
[150]	validation_0-auc:0.86007
[200]	validation_0-auc:0.86828
[250]	validation_0-auc:0.87198
[300]	validation_0-auc:0.87314
[350]	validation_0-auc:0.87308
[400]	validation_0-auc:0.87409
[450]	validation_0-auc:0.87478
[500]	validation_0-auc:0.87523
[550]	validation_0-auc:0.87600
[600]	validation_0-auc:0.87700
[650]	validation_0-auc:0.87660
[700]	validation_0-auc:0.87637
[712]	validation_0-auc:0.87656
test score 0.9010654688846526
val score 0.877288895897948
train score 0.9455943298390239


In [611]:
clf = CatBoostClassifier(iterations=200,
                         thread_count=-1,
                         random_seed=42,
                         eta = 0.3,
                         eval_metric='AUC',
                         l2_leaf_reg=5,
                         min_data_in_leaf=15,
                         subsample =0.9,
                         depth = 2
                         
                        )

clf.fit(
    X_train_normed,
    y_train,
    eval_set=(X_val_normed, y_val),
    verbose=50,
    use_best_model=True,
    early_stopping_rounds=50,
)


clf_test_preds = clf.predict_proba(X_test_normed)
clf_val_preds = clf.predict_proba(X_val_normed)
clf_train_preds = clf.predict_proba(X_train_normed)

print('test score', roc_auc_score(y_test, [i[1] for i in clf_test_preds]))
print('val score', roc_auc_score(y_val, [i[1] for i in clf_val_preds]))
print('train score', roc_auc_score(y_train, [i[1] for i in clf_train_preds]))

0:	test: 0.6966654	best: 0.6966654 (0)	total: 6.74ms	remaining: 1.34s
50:	test: 0.8624326	best: 0.8624326 (50)	total: 290ms	remaining: 847ms
100:	test: 0.8698184	best: 0.8700462 (97)	total: 577ms	remaining: 566ms
150:	test: 0.8696257	best: 0.8704726 (117)	total: 854ms	remaining: 277ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8704725858
bestIteration = 117

Shrink model to first 118 iterations.
test score 0.8942104301047188
val score 0.8704725858171687
train score 0.9287542688042064


In [626]:
xgb_test_preds.shape, clf_test_preds.shape

((555, 2), (555, 2))

In [632]:
X_test_new = np.concatenate((xgb_test_preds, clf_test_preds), axis=1)
X_train_new = np.concatenate((xgb_train_preds, clf_train_preds), axis=1)

In [634]:
y_train.shape, X_train_new.shape

((5545,), (5545, 4))

In [623]:
def cv_xgb(model, X, y, scoring='roc_auc', cv=5, **kwargs):
    kf = KFold(cv, shuffle=True)

    importances_ = []
    for train_index, valid_index in kf.split(X):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        model.fit(X_train, y_train, eval_set=[[X_valid, y_valid]], verbose=10000)
        
        xgb_val_preds = xgb.predict_proba(X_valid,
                              iteration_range=(0, xgb.best_iteration + 1))
        
        print('val score', roc_auc_score(y_valid, [i[1] for i in xgb_val_preds]))

    


xgb = XGBClassifier(
    eval_metric="auc",
    n_estimators=10000,
    random_state=42, 
    verbosity=0, 
    eta = 0.04, 
    max_depth = 3, 
    gamma = 0.9,
    min_child_weight=20,
    colsample_bytree=0.3,
    colsample_bynode=0.2,
    subsample =0.9,
    reg_lambda=0.6,
    # reg_alpha=0.3,
    early_stopping_rounds=100
)

cv_xgb(xgb, X, y)

[0]	validation_0-auc:0.61901
[978]	validation_0-auc:0.88100
val score 0.881242426040016
[0]	validation_0-auc:0.63022
[1088]	validation_0-auc:0.87783
val score 0.877888520366552
[0]	validation_0-auc:0.66330
[599]	validation_0-auc:0.88617
val score 0.8868027837910973
[0]	validation_0-auc:0.65080
[794]	validation_0-auc:0.90464
val score 0.904945260923988
[0]	validation_0-auc:0.59317
[930]	validation_0-auc:0.88594
val score 0.8863169996174922


In [638]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train_new, y_train)
test_ans = clf.predict_proba(X_test_new)
train_ans = clf.predict_proba(X_train_new)
print('test score', roc_auc_score(y_test, [i[1] for i in test_ans]))
print('train score', roc_auc_score(y_train, [i[1] for i in train_ans]))

test score 0.9002428642310605
train score 0.9461197312882708
